In [6]:
import sys
import pymol
import numpy as np
_stdouterr = sys.stdout, sys.stderr
pymol.finish_launching(['/usr/bin/pymol', '-q'])
sys.stdout, sys.stderr = _stdouterr

# load something into the PyMOL window
from pymol import cmd

CIF_FILES = '/home/vit/Projects/deeplife-project/data/cif_files'

In [ ]:
import vis_utils
sys.path.append('/home/vit/Projects/cryptoshow-analysis/src/B-evaluate-cryptoshow')
sys.path.append('/home/vit/Projects/cryptoshow-analysis/src')
import eval_utils
import cryptoshow_utils

CBS_DATA_PATH = '/home/vit/Projects/cryptoshow-analysis/data/A-cluster-ligysis-data/clustered-binding-sites.txt'
PREDICTIONS_PATH = '/home/vit/Projects/cryptoshow-analysis/data/D-visualize/predictions'

# load ground truth binding residues: these have mmcif numbering and need to be mapped to auth labeling
binding_residues_mmcifed, _ = eval_utils.read_test_binding_residues(data_path=CBS_DATA_PATH, pocket_types=['NON_CRYPTIC'])
binding_residues_mmcifed = vis_utils.reformat_binding_residues(binding_residues_mmcifed)

# load predictions
p2rank_predictions_mmcifed = vis_utils.read_predictions(data_path=f'{PREDICTIONS_PATH}/p2rank', protein_ids=binding_residues_mmcifed.keys())
model_predictions_mmcifed = vis_utils.read_predictions(data_path=f'{PREDICTIONS_PATH}/finetuning-with-smoothing', protein_ids=binding_residues_mmcifed.keys())

# map binding residues to auth labeling
binding_residues = {protein_id: [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket) for pocket in pockets] for protein_id, pockets in binding_residues_mmcifed.items()}
p2rank_predictions = {protein_id: [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket) for pocket in pockets] for protein_id, pockets in p2rank_predictions_mmcifed.items()}
model_predictions = {protein_id: [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket) for pocket in pockets] for protein_id, pockets in model_predictions_mmcifed.items()}

In [ ]:
from pymol.cgo import *

for protein_id in binding_residues.keys():
    this_binding_residues = np.concatenate(binding_residues[protein_id]) if binding_residues[protein_id] else np.array([])
    this_p2rank_predictions = np.concatenate(p2rank_predictions[protein_id]) if p2rank_predictions[protein_id] else np.array([])

    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.fetch(protein_id)
    cmd.color('grey', protein_id)
    if len(this_binding_residues) != 0:
        cmd.color('red', vis_utils.generate_pymol_algebra_selection(protein_id, this_binding_residues))
    else:
        continue
    if len(this_p2rank_predictions) != 0:
        cmd.color('blue', vis_utils.generate_pymol_algebra_selection(protein_id, this_p2rank_predictions))
        intersection = vis_utils.get_intersection(this_binding_residues, this_p2rank_predictions)
        if len(intersection) != 0:
            cmd.color('cyan', vis_utils.generate_pymol_algebra_selection(protein_id, intersection))

    DCCs = vis_utils.compute_DCCs(protein_id, binding_residues_mmcifed[protein_id], p2rank_predictions_mmcifed[protein_id])
    successful_predictions = vis_utils.count_successful_predictions(DCCs)
    print(f'{successful_predictions} / {len(binding_residues_mmcifed[protein_id])} pockets found. {DCCs}')
    
    # cmd.show('surface', protein_id)

    vis_utils.draw_pocket_centers(binding_residues_mmcifed[protein_id], p2rank_predictions_mmcifed[protein_id], protein_id)
    vis_utils.draw_pocket_center_lines(binding_residues_mmcifed[protein_id], p2rank_predictions_mmcifed[protein_id], protein_id)

    cmd.zoom(protein_id)

    user_input = input(">Press Enter for the next protein (press 'q' to quit)...\n")
    if user_input.lower() == 'q':
        break

NameError: name 'p2rank_predictions' is not defined